In [7]:
# NGSolve Libraries
from netgen.geom2d import unit_square
from ngsolve import *
from ngsolve.webgui import Draw # para jupyter
#import netgen.gui
from netgen.occ import *
from netgen.csg import *

import sys
sys.path.insert(0,"../")
import problems
import numpy as np
import params
import pickle
from time import time
from scipy.optimize import minimize_scalar

In [8]:
## get problem parameters and geometry
problem = problems.problem4

phi0 = problem[0]['phi0']
chi = problem[0]['chi']
G_target = problem[0]['G']
geom = "../"+problem[1]
dim = problem[0]['dim']
BC = problem[2]
name = problem[-1]
h = 1
ord = 3
N = params.N
KBTV = params.KBTV
form = "Functional" # EDP //functional

rho = 1.23e-6


phi = lambda J: phi0/J

G = Parameter(G_target)

## Generate mesh and geometry ### add parallel stuff
def mesher(geom, h):
    if ".stp" in geom:
        geo = OCCGeometry(geom)
    else:
        geo = pickle.load(open(geom, "rb"))

    mesh = Mesh(geo.GenerateMesh(maxh=h))
    return mesh
mesh = mesher(geom, h)  

def Gel_energy_functional(F,u):
    g = 9.81
    J = Det(F)
    phi = phi0/J
    H = (J - phi0)*log(1-phi)  + phi0 * chi*(1-phi)
    # calculate C
    # min (G/2) * (3 lam**2) + KBTV * H(lam**3):  lam > 1
    Hf = lambda lam: (lam-phi0)*np.log(1-phi0/lam) + phi0*chi*(1-phi0/lam)

    fun = lambda lam: (G.Get()/2)*(3*lam**2) + KBTV*Hf(lam)
    res = minimize_scalar(fun, bounds=(1, 10), method='bounded')
    C = res.fun
    return 0.5*(G)* Trace(F.trans*F ) + H*KBTV
    return 0.5*(G)* Trace(F.trans*F ) + H*KBTV + rho*g*u[1] - C


def Gel_energy_EDP(F): ## |F|^2 + H => gamma F:Gradv + H'*J'
    # ddet(A(t))/dt = det(A(t))*trace(A^-1(t)*Grad (v))
    
    J = Det(F)
    phi = phi0/J
    dv = Grad(v)
    invF = Inv(F)
    H_prime = -phi/N + log(1-phi) + phi + chi*phi**2
    edp = (G.Get()/KBTV) * InnerProduct(F,dv) + H_prime * J * Trace(invF*dv)
    return edp


## Generate spaces and forms
"""
To bond the gel go to geometries and describe the bonding
face there, not here.
"""
if BC["dir_cond"] == "faces":
    fesu = VectorH1(mesh, order=ord, dirichlet = BC["DIR_FACES"])
elif BC["dir_cond"] == "components":
    fesu = VectorH1(mesh, order=ord, dirichletx = BC["x"], dirichlety = BC["y"], dirichletz = BC["z"])

fesphi = L2(mesh, order = ord-2)

fes = FESpace([fesu, fesphi])
u, delta = fes.TrialFunction()
v, vphi= fes.TestFunction()
BF = BilinearForm(fes)


## Assemble forms
alpha = Parameter(1)

psih = GridFunction(fesphi)
psih.Set(0)

uk = GridFunction(fesu)
uk.Set((0,0))
psik = GridFunction(fesphi)
eps = 1e-6
# define phi as the 0 function 

gfu = GridFunction(fes)
gfu.vec[:] = 0
obs = CF(-y)    

def Assemble_BF(BF,G_funs,form):
    u, v, alpha, delta, psih, psik = G_funs
    F = Id(2)+ grad(u)
    if form == "Functional":
        #LHS
        
        BF += Variation(alpha * Gel_energy_functional(F,u).Compile()*dx)
        BF += delta * v[1]*dx
        BF += u[1]*vphi*dx - (delta * exp(psih) * vphi * dx + eps * (grad(delta) * grad(vphi) * dx))
        #RHS
        BF += -(psik - psih)*v[1]*dx
        BF += -(phi + exp(psih))*vphi*dx
        return BF
    elif form == "EDP":
        BF += alpha * Gel_energy_EDP(F).Compile() * dx
        BF += delta * v[1]*dx
        BF += u[1]*vphi*dx - (delta * exp(psih) * vphi * dx + eps * (grad(delta) * grad(vphi) * dx))
        #RHS
        BF += -(psik - psih)*v[1]*dx
        BF += -(obs + exp(psih))*vphi*dx
        return BF
    






In [9]:
max_PG_it = 10
max_iter_qnewton = 20
max_iter_newton = 30
newton_damp = 0.5
softening_n = 15

tol_newton = 1e-6
tol_QN = 1e-5
tol_PG = 1e-5

gammas = np.flip(np.linspace(G_target, G_target*20 ,softening_n))


for num, load in enumerate(gammas):
    G.Set(load) #incremental softening
    if num == softening_n-1:
        max_PG_it = 20
    for k in range(1,max_PG_it):

        print("PG it:",k)
        alpha.Set(2**k)
        psik.vec.data = psih.vec
        if k != 0:
            uk.vec.data = gfu.components[0].vec # update u_k to compare later
        print("Starting QN")
        for i in range(max_iter_qnewton):
            G_funs = u,v,alpha,delta,psih,psik 
            with TaskManager():
                BF = Assemble_BF(BF,G_funs,form) # get bi(not)linear form
            
            
            res = gfu.vec.CreateVector()
            w = gfu.vec.CreateVector()
            #iterations of newton for the NL problem at each QN step
            print("Solving nonlinear subproblem")
            for iter in range(max_iter_newton):
                with TaskManager():
                    BF.Apply(gfu.vec,res)
                    BF.AssembleLinearization(gfu.vec)
                    inv = BF.mat.Inverse(freedofs= fes.FreeDofs())
                    w.data = newton_damp * inv * res
                    gfu.vec.data += -w
                if abs(InnerProduct(w,res)) < tol_newton:
                    print("Tol achieved", InnerProduct(w,res))
                    break
            # check tolerance on QN
            qn_tol_val= Integrate((gfu.components[1])**2,mesh)
            if qn_tol_val < tol_QN:
                print("QN tol achieved",qn_tol_val )
                break
            # update psih
            psih.vec.data = psih.vec + gfu.components[1].vec
        # check error on gfu
        PG_error = Integrate((uk-gfu.components[0])**2,mesh)
        if PG_error < tol_PG:
            print("PG tol achieved",PG_error)
            break
        print("Error",PG_error)

    Draw(gfu.components[0],mesh)



PG it: 1
Starting QN
Solving nonlinear subproblem
Tol achieved -6.659225059646236e-07
Solving nonlinear subproblem
Tol achieved -9.455888192069505e-07
Solving nonlinear subproblem
Tol achieved -6.299687225322044e-07
Solving nonlinear subproblem
Tol achieved -6.78577359030599e-07
Solving nonlinear subproblem
Tol achieved -6.663403588481226e-07
Solving nonlinear subproblem
Tol achieved -7.533306062141632e-07
Solving nonlinear subproblem
Tol achieved -6.766768090608266e-07
Solving nonlinear subproblem
Tol achieved -3.4975351370696425e-07
Solving nonlinear subproblem
Tol achieved -5.894592092670473e-07
Solving nonlinear subproblem
Tol achieved -7.493152301296573e-07
QN tol achieved 6.2875239134566456e-09
Error 100.42004264517062
PG it: 2
Starting QN
Solving nonlinear subproblem
Tol achieved 4.667145070958795e-07
Solving nonlinear subproblem
Tol achieved -4.5295716116557077e-07
Solving nonlinear subproblem
Tol achieved -3.4886311710123737e-07
Solving nonlinear subproblem
Tol achieved -6.822

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

PG it: 1
Starting QN
Solving nonlinear subproblem
Tol achieved 8.50603211599022e-07
Solving nonlinear subproblem
Tol achieved -2.9216913073260676e-07
QN tol achieved 7.884998492452187e-06
Error 1.1267208190112794
PG it: 2
Starting QN
Solving nonlinear subproblem
Tol achieved 4.0471772842499233e-07
Solving nonlinear subproblem
Tol achieved -7.659603621332028e-07
QN tol achieved 1.5052545192003897e-06
Error 0.021648846142656816
PG it: 3
Starting QN
Solving nonlinear subproblem
Tol achieved 7.46520460911901e-07
Solving nonlinear subproblem
Tol achieved -3.627324996494947e-07
QN tol achieved 1.9289076682127354e-06
Error 0.006800138725462121
PG it: 4
Starting QN
Solving nonlinear subproblem
Tol achieved 5.959264104214013e-07
Solving nonlinear subproblem
Tol achieved -9.352559370095591e-07
QN tol achieved 1.1581935359793784e-06
Error 0.00409774422848287
PG it: 5
Starting QN
Solving nonlinear subproblem
Tol achieved 9.767278485506637e-07
Solving nonlinear subproblem
Tol achieved -9.9637483299

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

PG it: 1
Starting QN
Solving nonlinear subproblem
Tol achieved 3.105734885985492e-07
Solving nonlinear subproblem
Tol achieved -4.432407260680915e-07
Solving nonlinear subproblem
Tol achieved -3.690286337466061e-07
QN tol achieved 6.031405757821507e-09
Error 1.337547594183497
PG it: 2
Starting QN
Solving nonlinear subproblem
Tol achieved 6.509597616675909e-07
Solving nonlinear subproblem
Tol achieved -3.3140843103890343e-07
QN tol achieved 3.642652972488e-06
Error 0.027233785627471512
PG it: 3
Starting QN
Solving nonlinear subproblem
Tol achieved 3.3040488287394696e-07
Solving nonlinear subproblem
Tol achieved -6.752826189429553e-07
QN tol achieved 7.365305646520939e-06
Error 0.008747843219079345
PG it: 4
Starting QN
Solving nonlinear subproblem
Tol achieved 3.189030743501288e-07
Solving nonlinear subproblem
Tol achieved -5.522916379617869e-07
QN tol achieved 6.813404930634984e-06
Error 0.006708478501176409
PG it: 5
Starting QN
Solving nonlinear subproblem
Tol achieved 6.96640234290556

...

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

PG it: 1
Starting QN
Solving nonlinear subproblem
Tol achieved 4.5207268779639e-07
Solving nonlinear subproblem
Tol achieved -2.987878838550021e-07
Solving nonlinear subproblem
Tol achieved -4.63802587105308e-07
QN tol achieved 4.4451969299180494e-08
Error 114.53384694595215
PG it: 2
Starting QN
Solving nonlinear subproblem
Tol achieved 3.465607801678618e-07
Solving nonlinear subproblem
Tol achieved -4.067612278025555e-07
Solving nonlinear subproblem
Tol achieved -2.8291846411278957e-07
QN tol achieved 9.57999371978368e-09
Error 12.497887860510001
PG it: 3
Starting QN
Solving nonlinear subproblem
Tol achieved 3.063784949851379e-07
Solving nonlinear subproblem
Tol achieved -9.172109680934259e-07
Solving nonlinear subproblem
Tol achieved -6.879651710644382e-07
QN tol achieved 1.2644585532371767e-07
Error 1.7828502661994943
PG it: 4
Starting QN
Solving nonlinear subproblem
Tol achieved 3.997455016346264e-07
Solving nonlinear subproblem
Tol achieved -6.545727809430471e-07
Solving nonlinear

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

PG it: 1
Starting QN
Solving nonlinear subproblem
Tol achieved 3.683002134720252e-07
Solving nonlinear subproblem
Tol achieved -7.853724438083647e-07
Solving nonlinear subproblem
Tol achieved -5.016892974703429e-07
QN tol achieved 5.025255676343851e-08
Error 630.8613620132634
PG it: 2
Starting QN
Solving nonlinear subproblem
Tol achieved 3.23284420378169e-07
Solving nonlinear subproblem
Tol achieved -5.315598180880231e-07
Solving nonlinear subproblem
Tol achieved -8.857177043616871e-07
QN tol achieved 5.719649255595327e-07
Error 71.96249674089378
PG it: 3
Starting QN
Solving nonlinear subproblem
Tol achieved 6.515360705986493e-07
Solving nonlinear subproblem
Tol achieved -7.563094187683616e-07
Solving nonlinear subproblem
Tol achieved -7.478268720214769e-07
QN tol achieved 8.405152157414154e-08
Error 25.787997401670793
PG it: 4
Starting QN
Solving nonlinear subproblem
Tol achieved 5.592266990651288e-07
Solving nonlinear subproblem
Tol achieved -4.126823676816549e-07
Solving nonlinear s

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…